In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.utils import resample
from sklearn.grid_search import GridSearchCV
import re
import xgboost as xgb

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [13]:
data = pd.read_csv("train.csv")
# df_majority = data[data.Label==0]
# df_minority = data[data.Label==1]
# df_minority_upsampled = resample(df_minority, 
#                                  replace=True,     # sample with replacement
#                                  n_samples=df_majority.shape[0],    # to match majority class
#                                  random_state=123) # reproducible results
# df_majority_downsampled = resample(df_majority,
#                                   replace=True,
#                                   n_samples=df_minority.shape[0],
#                                   random_state=123)
# data = pd.concat([df_majority, df_minority_upsampled])
# data
# size = 3000
# a = resample(df_minority,
#              replace=True,
#              n_samples=size,
#              random_state=123)
# b = resample(df_majority_downsampled,
#             replace=True,
#             n_samples=size,
#             random_state=123)
# data = pd.concat([df_minority, df_majority_downsampled])
# data = pd.concat([a,b])
data = data.sample(frac=1)
data = data.reset_index(drop=False).drop(["index"], axis=1)



In [5]:
vowel = u'уеёыаоэяию'
consonant = u'йцкнгшщзхъфвпрлджчсмтьб'
ru_letter = vowel + consonant + u"-’'"

def count_vow(word):
    vow = 0
    for i in word:
        if i in vowel:
            vow += 1
    return vow

def count_con(word):
    cons = 0
    for i in word:
        if i in consonant:
            cons += 1
    return cons

def is_only_ru(word):
    return len(set(word) - set(ru_letter)) == 0

data["Word"] = data["Word"].apply(lambda x: x.lower())
data = data[data["Word"].apply(lambda x: is_only_ru(x))==1]


last_letter = list(set(data["Word"].apply(lambda x: x[-1]).get_values()))
last_2_letter = list(set(data["Word"].apply(lambda x: x[-2:]).get_values()))
last_3_letter = list(set(data["Word"].apply(lambda x: x[-3:]).get_values()))


def last_letters(data):
    for letter in  last_letter:
        data["last1" + letter] = data["Word"].apply(lambda x: x[-1] == letter)
    for letter in last_2_letter:
        data["last2" + letter] = data["Word"].apply(lambda x: x[-2:] == letter)
    for letter in last_3_letter:
        data["last2" + letter] = data["Word"].apply(lambda x: x[-3:] == letter)
    return data

In [6]:
def add_features(data):
    data["len"] = data["Word"].apply(lambda x: len(x))
    data["is_len_more_11"] = data["len"].apply(lambda x: x > 11)
    data["vow_count"] = data["Word"].apply(lambda x: count_vow(x))
    data["cons_count"] = data["Word"].apply(lambda x: count_con(x))
    data["last_vow"] = data["Word"].apply(lambda x: x[-1] in vowel)
    data["prelast_vow"] = data["Word"].apply(lambda x: len(x) > 2 and x[-2] in vowel)
    data["ohara"] = data["Word"].apply(lambda x: x[:2] == "о'")
    data = last_letters(data)
    return data

In [14]:
data = add_features(data)
X = data.drop(["Label", "Word"], axis=1)
y = data["Label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# regr = xgb.XGBClassifier(colsample_bytree=0.97, max_depth=10, n_estimators=90, subsample=0.97)
# regr = RandomForestClassifier(max_depth=160, n_estimators=45)
classifier = SGDClassifier(penalty='l1', class_weight='auto')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(roc_auc_score(y_pred, y_test.get_values()))
# 0.754698266885

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


ValueError: class_weight must be dict, 'balanced', or None, got: 'auto'

In [65]:
# classifier = xgb.XGBClassifier()
# # {'max_depth': 11, 'subsample': 0.9}
# # {'colsample_bytree': 0.95, 'max_depth': 10, 'n_estimators': 80, 'subsample': 0.97}
# # 0.7825313337289386
# params = {
#     'max_depth': [9, 10, 11],
#     'subsample': [0.97, 0.99, 1],
#     'n_estimators': [70, 80, 90],
#     'colsample_bytree': [0.95, 0.97, 1]
# }
# grid_search = GridSearchCV(classifier, params, n_jobs=-1, verbose=1, scoring='roc_auc')
# grid_search.fit(X, y)
# print(grid_search.best_params_)
# print(grid_search.best_score_)

In [9]:
classifier = SGDClassifier()
# {'max_depth': 100, 'max_features': 'sqrt', 'n_estimators': 50, 'n_jobs': -1}
# 0.8426727676293007
# {'max_depth': 150, 'n_estimators': 45, 'n_jobs': -1}
# 0.8427648816047106
params = {
    'penalty': ['l2', 'l1'],
    
#     'max_depth': [130, 140, 150, 160, 170],
#     'n_estimators' : [42, 45, 47, 50, 52],
    'n_jobs' : [-1]
}
grid_search = GridSearchCV(classifier, params, n_jobs=-1, verbose=1, scoring='roc_auc')
grid_search.fit(X, y)
print(grid_search.best_params_)
print(grid_search.best_score_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/s

{'n_jobs': -1, 'penalty': 'l1'}
0.7988240240825243


In [67]:
test = pd.read_csv("test.csv")
test["Word"] = test["Word"].apply(lambda x: x.lower())
test = add_features(test).drop(["Word"], axis=1)
pred = regr.predict(test)
# test
pred

array([0, 0, 1, ..., 0, 0, 0])

In [68]:
pd.Series(pred).to_csv("submission_random_forest_last2.csv")